**Script Overview**

**1. Situation:**

This script matches IATA codes / OAG ac names and Planespotters ac.

There is a general difference between the IATA and the ICAO aircraft codes: IATA has designated, superordinated code for "all models" while the ICAO has not. (In the following, we call these aircraft types "IATA-only".)

This becomes clear at the following example:
- IATA=141 for B461 BAe 146-100 Pax --> ICAO=B464
- IATA=142 for B462 BAe 146-200 Pax --> ICAO=B462
- IATA=143 for B463	BAe 146-300 Pax --> ICAO=B463
- IATA=146 for BAe 146 all pax models --> ICAO=N/A <- this is the general, superordinated IATA code.

**2. Problem**: 

This OAG data set is based on the IATA code, but the Planespotters (PS) data (data on aircraft, including e.g. age and seat capacity) does always have specific AC models, not the general, superordinated AC.

**3. Solution**:

We distribute the OAG flights performed by the IATA-only AC types to all specific IATA-code AC that are available in the set. We do so per route and according to the share of flight kilometers observed in the OAG data.

Example for 1 specific route:
- IATA=146: 10 flights & 2000 FKM in OAG data set --> 0 FKM after reassignment
- IATA=141: 20 flights & 4000 FKM in OAG data set --> 4800 FKM after reassignment
- IATA=142: 30 flights & 6000 FKM in OAG data set --> 7200 FKM after reassignment
The 2000 FKM from IATA=146 are reassigned to the 141 and 142 AC according to the share of FKM in OAG.

## Load packages

In [1]:
import pandas as pd
import numpy as np
import math

## Load OAG data

In [2]:
# Connect to DB
%load_ext ipython_pg.LAV
%pg_connect dbname='mobility' host='lav-fileserver.ethz.ch' user='heldm'

password for heldm@lav-fileserver.ethz.ch:5433:········
SUCCESS: connected to lav-fileserver.ethz.ch
  PostGIS integration enabled

In [3]:
%%pg_pd oag18
SELECT * FROM "OAG"."2018_schedule"

SUCCES: matched 180578 rows
 results stored as 'oag18'


In [4]:
oag18.head()

,ac_code_iata,ac_name_oag,arr_ap_code,arr_ap_name,arr_ap_cty,dep_ap_code,dep_ap_name,dep_ap_cty,gc_distance,frequency_2018,fkm_2018
0,100,Fokker 100,ADE,Aden,YE,JIB,Djibouti,DJ,248.203212,58.0,14395.786303
1,100,Fokker 100,AMS,Amsterdam,NL,BRU,Brussels Airport,BE,157.553624,88.0,13864.718918
2,100,Fokker 100,AMS,Amsterdam,NL,MUC,Munich International Airport,DE,676.777570,332.0,224690.153347
3,100,Fokker 100,AZD,Yazd,IR,BND,Bandar Abbas,IR,559.459639,91.0,50910.827163
4,100,Fokker 100,BND,Bandar Abbas,IR,AZD,Yazd,IR,559.459639,91.0,50910.827163


## Make OAG data set Planespotters-compatible

### Reassign generic IATA Codes

**Load manual IATA-ICAO lookup table**

In [5]:
# Read the manual lookup table
ac_type_designators = pd.read_excel('../../../../Research/Aviation/Jupyter/datasets/aircraft_type_designators_manual.xlsx')
ac_type_designators = ac_type_designators.rename(columns={'oag_model: all generic AC types where we have multiple variants in the PS data set are marked in red': 'ac_name_oag',
                                            'planespotters_model': 'ac_name_ps',
                                            'share of yearly flight km: high impact models (>1% of total FKM) are marked in green -> ask OAG here': 'share_of_yearly_FKM'})
ac_type_designators = ac_type_designators.drop(['share_of_yearly_FKM','ps_model assignment','comment1','comment2', 'widebody'], axis=1)
ac_type_designators['ac_code_iata_reassign'] = ac_type_designators['ac_code_iata_reassign'].str.split(',')
ac_type_designators['ac_code_iata'] = ac_type_designators['ac_code_iata'].astype(str)
ac_type_designators.head()

,ac_code_iata,ac_code_icao,ac_name_oag,ac_name_ps,ac_code_iata_reassign,oag_name_keep
0,CS1,BCS1,Airbus A220-100,Airbus A220-100,NaN,NaN
1,CS3,BCS3,Airbus A220-300,Airbus A220-300,NaN,NaN
2,AB3,A30B,Airbus A300 Passenger,Airbus A300B4,NaN,no
3,AB6,A306,Airbus A300-600 Passenger,Airbus A300-600,NaN,NaN
4,AB4,A30B,Airbus A300B2/B4 Passenger,Airbus A300B4,NaN,Airbus A300B2/B4 Passenger


In [6]:
# Extract all iata codes that have to be reassigned
reassign_df = ac_type_designators[ac_type_designators['ac_name_ps'] == 'to_be_assigned_to_multiple_variants'].reset_index()
reassign_df

,index,ac_code_iata,ac_code_icao,ac_name_oag,ac_name_ps,ac_code_iata_reassign,oag_name_keep
0,8,32S,to_be_assigned_to_multiple_variants,Airbus A318/319/320/321,to_be_assigned_to_multiple_variants,"[318, 319, 320, 32A, 321, 32B]",NaN
1,16,330,to_be_assigned_to_multiple_variants,Airbus A330,to_be_assigned_to_multiple_variants,"[332, 333]",NaN
2,20,340,to_be_assigned_to_multiple_variants,Airbus A340,to_be_assigned_to_multiple_variants,"[342, 343, 345, 346]",NaN
3,25,350,to_be_assigned_to_multiple_variants,Airbus A350,to_be_assigned_to_multiple_variants,"[351, 359]",NaN
4,40,ATR,to_be_assigned_to_multiple_variants,ATR42 /ATR72,to_be_assigned_to_multiple_variants,"[AT4, AT7]",NaN
5,42,ARJ,to_be_assigned_to_multiple_variants,Avro RJ70 /rj85 /rj100,to_be_assigned_to_multiple_variants,"[AR1, AR8]",NaN
6,44,146,to_be_assigned_to_multiple_variants,BAe 146 Passenger,to_be_assigned_to_multiple_variants,"[141, 142, 143]",NaN
7,62,M80,to_be_assigned_to_multiple_variants,Boeing (douglas) MD-80,to_be_assigned_to_multiple_variants,"[M81, M82, M83, M87, M88]",NaN
8,73,737,to_be_assigned_to_multiple_variants,Boeing 737 Passenger,to_be_assigned_to_multiple_variants,"[732, 73C, 733, 734, 73E, 735, 736, 73W, 73G, ...",NaN
9,98,757,to_be_assigned_to_multiple_variants,Boeing 757 (Passenger),to_be_assigned_to_multiple_variants,"[752, 753]",NaN


**Reassign**

In [7]:
# Add the total share of yearly fkm of each OAG entry and a one-string-route as two new columns
oag18['share_of_fkm_2018'] = oag18['fkm_2018'] / oag18['fkm_2018'].sum()
oag18['route'] = oag18['arr_ap_code'] + '_' + oag18['dep_ap_code']
oag18.head()

,ac_code_iata,ac_name_oag,arr_ap_code,arr_ap_name,arr_ap_cty,dep_ap_code,dep_ap_name,dep_ap_cty,gc_distance,frequency_2018,fkm_2018,share_of_fkm_2018,route
0,100,Fokker 100,ADE,Aden,YE,JIB,Djibouti,DJ,248.203212,58.0,14395.786303,2.767831e-07,ADE_JIB
1,100,Fokker 100,AMS,Amsterdam,NL,BRU,Brussels Airport,BE,157.553624,88.0,13864.718918,2.665724e-07,AMS_BRU
2,100,Fokker 100,AMS,Amsterdam,NL,MUC,Munich International Airport,DE,676.777570,332.0,224690.153347,4.320044e-06,AMS_MUC
3,100,Fokker 100,AZD,Yazd,IR,BND,Bandar Abbas,IR,559.459639,91.0,50910.827163,9.788459e-07,AZD_BND
4,100,Fokker 100,BND,Bandar Abbas,IR,AZD,Yazd,IR,559.459639,91.0,50910.827163,9.788459e-07,BND_AZD


In [8]:
def lookup_fkm_shares(iata_orig, ls_reassign, oag18_route, oag18):
    '''Take the original generic iata code aircraft (iata_orig) that has to be reassigned to other aircraft types,
    which are the input list ls_reassign aircraft types.
    Takes the oag data for a specific route - all aircraft that perform these route.
    Returns relative shares of these to-be-reassigned aircraft types.'''
    
    ls_reassign = pd.DataFrame({'ac_code_iata': ls_reassign, 'share_of_fkm_2018': 'share_of_fkm_2018'})
    ac_performance = oag18.groupby('ac_code_iata').sum().reset_index()
    ls_reassign['share_of_fkm_2018'] = ac_performance.set_index('ac_code_iata').lookup(ls_reassign['ac_code_iata'],ls_reassign['share_of_fkm_2018'])
    ls_reassign['share_of_fkm_2018'] = ls_reassign['share_of_fkm_2018'] / ls_reassign['share_of_fkm_2018'].sum()
    
    return ls_reassign

In [9]:
new_oag18 = oag18.copy()

In [10]:
'''Some iata codes in the OAG data set do not refer to a specific aircraft, but are a general "melting pot" for all aircraft versions of a general type.
Since stock data from Planespotters does refer only to specific aircraft, we have to reassign the general aircraft types to the specific ones.
This code block assures that this reassignment (redistribution of number of flights (frequency_2018) and flight kilometers (fkm_2018)) is done according to 
the share of specific aircraft versions on the total fkm in 2018. Since fkm correlate good with number of aircraft in a first order approximation,
yearly fkm share is a good proxy for the number of aircraft version that we reassign.'''

for iata_orig in np.arange(len(reassign_df)): # loop over all iata codes that have to be replaced (iata_orig = [0,1,2,...,#all orig iata codes])
    
    orig = reassign_df.loc[iata_orig,'ac_code_iata']           # original ac that has to be reassigned (e.g. '330')
    reass = reassign_df.loc[iata_orig,'ac_code_iata_reassign'] # ac to which the fkm are reassigned (e.g. ['332', '333'])
    
    
    for route in oag18[oag18['ac_code_iata'] == orig]['route']: # loop over routes
        
        if len(oag18[(oag18['route'] == route) & (oag18['ac_code_iata'] == orig)]) > 0: # check if orig ac is operated at the current route
            # Look sequentially at routes in the oag data for the orig aircraft
            oag18_route = oag18[(oag18['route'] == route) & \
                                ((oag18['ac_code_iata'] == orig) | (oag18['ac_code_iata'].isin(reass)))].reset_index()
            oag18_route = oag18_route.drop(columns='index')

            if len(oag18_route) > 0:
                # Create a new table for all reass aircraft and set empty cells that are filled afterwards
                orig_oag18_route = oag18_route[oag18_route['ac_code_iata'] == orig]
                new_df = pd.concat([orig_oag18_route]*len(reass), ignore_index=True)
                new_df.at[:,['ac_code_iata', 'ac_name_oag','frequency_2018', 'fkm_2018', 'share_of_fkm_2018']] = ''
                new_df.at[:,'ac_code_iata'] = reass
                new_df.at[:,'ac_name_oag'] = 'new_reass_ac'

                # Define shares of yearly fkm shares of the reass aircraft
                shares = lookup_fkm_shares(orig, reass, oag18_route, oag18)

                # Calculate the fkm that have to be redistributed from the orig ac to the reass ac
                orig_fkm = oag18_route[oag18_route['ac_code_iata'] == orig]['fkm_2018'].item()


                # Loop over all reass ac for the specific aircraft orig (loop 1) and the specific route (loop 2)
                for i in reass:
                    # Get the share of fkm of i-th reass ac on the total fkm of all reass ac
                    share_i = shares[shares['ac_code_iata'] == i]['share_of_fkm_2018'].item()
            
                    # Check whether the i-th reass aircraft is already assigned for that route in the original oag data set and save the fkm of it if it is nonzero.
                    reass_fkm_old = oag18_route[oag18_route['ac_code_iata'] == i]['fkm_2018']
                    if len(reass_fkm_old) >0:
                        reass_fkm_old = reass_fkm_old.item()
                    else:
                        reass_fkm_old = 0
                    
                    # Fill the dataframe for the reass aircraft with the share-wise distribution of the to-be-reassigned orig fkm + the reass_fkm_old
                    row = new_df[new_df['ac_code_iata'] == i].index.item()
                    
                    
                    new_df.at[row,'fkm_2018'] = share_i*orig_fkm + reass_fkm_old
                    # Calculate the frequency from the fkm and the great-circle distance
                    new_df.at[row,'frequency_2018'] = round(new_df.loc[row,'fkm_2018'] / new_df.loc[row,'gc_distance'])   


                # Since we assign the fkm according to yearyl fkm shares, the frequency for a specific route-ac pair might not be a multiple of the gc_distance (see last step in reass-loop).
                # Solution: Make sure that the sum of frequency_2018 over all reass ac is the same as for all ac (orig + reass) on the route.
                diff_freq = new_df['frequency_2018'].sum() - oag18_route['frequency_2018'].sum() 

                # If there is a difference, add or substract that difference in number of flights from the reass ac with the largest freq,
                # unless there is no reass ac with a largest freq, then take the last ac in the list
                idx_max = pd.to_numeric(new_df['frequency_2018'])

                if abs(diff_freq) > 0:
                    if math.isnan(idx_max.idxmax()):
                        last_row = new_df.index[-1]
                        new_df.at[last_row,'frequency_2018'] -= diff_freq
                    else:
                        new_df.at[idx_max.idxmax(),'frequency_2018'] -= diff_freq

                # After this correction, we can reset the fkm's according to the frequencies:
                new_df.at[:,'fkm_2018'] = new_df.loc[:,'frequency_2018'] * new_df.loc[:,'gc_distance']

                # Now we append the rows with the new values for the reass ac and drop the original ones if existing
                new_oag18 = new_oag18.append(new_df, ignore_index=True)
                new_oag18 = new_oag18.drop_duplicates(subset = ['ac_code_iata','route'], keep = 'last')
                
    # Last we delete all orig ac
    idx_to_del = new_oag18[new_oag18['ac_code_iata'] == orig].index
    new_oag18 = new_oag18.drop(index=idx_to_del)

In [11]:
# Since the fkm-frequency-step in the loop above creates a few 0-frequency-entries, we delete them.
zeros = new_oag18[new_oag18['frequency_2018'] == 0].index
new_oag18 = new_oag18.drop(index=zeros)
new_oag18 = new_oag18.reset_index()
new_oag18 = new_oag18.drop(columns='index')

**Check whether the reassignment was successful: None of the to-be-reassigned-ac-codes should be in the df anymore:**

In [12]:
ac_code_iata_reassigned = list(reassign_df.ac_code_iata)
ac_code_iata_in_df = list(new_oag18.ac_code_iata.unique())
set(ac_code_iata_in_df).intersection(set(ac_code_iata_reassigned))

set()

**Sanity check:**

Frequency and fkm should be the same for new_oag18 and the entries of oag18:

In [13]:
check_diff_freq = abs(oag18['frequency_2018'].sum() - new_oag18['frequency_2018'].sum())
check_diff_fkm = abs(oag18['fkm_2018'].sum() - new_oag18['fkm_2018'].sum())
print('Difference between frequency before and after ac reassignment: {:.2f}.'.format(check_diff_freq))
print('Difference between total fkm before and after ac reassignment: {:.2f}.'.format(check_diff_fkm))

Difference between frequency before and after ac reassignment: 0.00.
Difference between total fkm before and after ac reassignment: 0.00.


Check passed.

** Since we split the to-be-reassigned ac codes into multiple other ac, we also increased the number of entries: **

In [14]:
len(oag18)

180578

In [15]:
len(new_oag18)

218729

### Merge duplicate ICAO codes

**Load manual IATA-ICAO lookup table**

In [16]:
# Read the manual lookup table
ac_type_designators = pd.read_excel('../../../../Research/Aviation/Jupyter/datasets/aircraft_type_designators_manual.xlsx')
ac_type_designators = ac_type_designators.rename(columns={'oag_model: all generic AC types where we have multiple variants in the PS data set are marked in red': 'ac_name_oag',
                                            'planespotters_model': 'ac_name_ps',
                                            'share of yearly flight km: high impact models (>1% of total FKM) are marked in green -> ask OAG here': 'share_of_yearly_FKM'})
ac_type_designators = ac_type_designators.drop(['share_of_yearly_FKM','ps_model assignment','comment1','comment2', 'widebody'], axis=1)
ac_type_designators['ac_code_iata_reassign'] = ac_type_designators['ac_code_iata_reassign'].str.split(',')
ac_type_designators['ac_code_iata'] = ac_type_designators['ac_code_iata'].astype(str)
ac_type_designators.head()

,ac_code_iata,ac_code_icao,ac_name_oag,ac_name_ps,ac_code_iata_reassign,oag_name_keep
0,CS1,BCS1,Airbus A220-100,Airbus A220-100,NaN,NaN
1,CS3,BCS3,Airbus A220-300,Airbus A220-300,NaN,NaN
2,AB3,A30B,Airbus A300 Passenger,Airbus A300B4,NaN,no
3,AB6,A306,Airbus A300-600 Passenger,Airbus A300-600,NaN,NaN
4,AB4,A30B,Airbus A300B2/B4 Passenger,Airbus A300B4,NaN,Airbus A300B2/B4 Passenger


**Remove generic IATA codes**

In [17]:
# We delete all generic IATA codes. They are reassigned to specific aircraft 
# variants in the Jupyter Notebook "OAG_Planespotters_Alignment".
ac_type_designators_wo_generic = ac_type_designators[ac_type_designators['ac_code_icao'] != 'to_be_assigned_to_multiple_variants']
ac_type_designators_wo_generic = ac_type_designators_wo_generic.drop(columns='ac_code_iata_reassign')
ac_type_designators_wo_generic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173 entries, 0 to 189
Data columns (total 5 columns):
ac_code_iata     173 non-null object
ac_code_icao     173 non-null object
ac_name_oag      173 non-null object
ac_name_ps       127 non-null object
oag_name_keep    66 non-null object
dtypes: object(5)
memory usage: 8.1+ KB


**Add ICAO code to OAG data.**

In [18]:
iata_icao_lookup = ac_type_designators_wo_generic.set_index('ac_code_iata').to_dict()['ac_code_icao']
new_oag18['ac_code_icao'] = new_oag18['ac_code_iata'].replace(iata_icao_lookup)

** Merge duplicate ICAO values**

In [19]:
# Define duplicate icao values
icao_dupl = ac_type_designators_wo_generic[ac_type_designators_wo_generic.duplicated(subset ='ac_code_icao', keep=False)].sort_values(by='ac_code_icao')
icao_dupl[6:10]

,ac_code_iata,ac_code_icao,ac_name_oag,ac_name_ps,oag_name_keep
13,321,A321,Airbus A321,Airbus A321-200,Airbus A321 (with and without Sharklets)
14,32B,A321,Airbus A321 (Sharklets),Airbus A321-200,no
28,380,A388,Airbus A380 Passenger,Airbus A380-800,no
29,388,A388,Airbus A380-800 Passenger,Airbus A380-800,Airbus A380-800 Passenger


Example: The A321 are all given the oag name "Airbus A321 (with and without Sharklets)" and the IATA code '321'.

In [20]:
icao_dupl_names = icao_dupl['ac_code_icao'].unique() # unique icao values in the list
icao_dupl_groups = icao_dupl.groupby('ac_code_icao')

for i in icao_dupl_names: # loop over all duplicate icao values
     # Get all entries from icao_dupl for the selected icao code
    _cache = icao_dupl_groups.get_group(i)
    
    # Rename the ac_name_oag to the one given in icao_dupl
    _rename_oag_name = _cache[_cache['oag_name_keep'] != 'no']['oag_name_keep'].item()
    _keep_iata_codes = _cache[_cache['oag_name_keep'] != 'no']['ac_code_iata'].to_list()
    _rename_iata_index = new_oag18[new_oag18['ac_code_iata'].isin(_keep_iata_codes)].index
    new_oag18.at[_rename_iata_index,'ac_name_oag'] = _rename_oag_name
    
    # Rename the ac_code_iata to the first one in the list 
    _del_iata_codes = _cache[_cache['oag_name_keep'] == 'no']['ac_code_iata'].to_list()
    _del_iata_index = new_oag18[new_oag18['ac_code_iata'].isin(_del_iata_codes)].index
    new_oag18.at[_del_iata_index,'ac_code_iata'] = _keep_iata_codes[0]

Now that we set common names for the same ac_code_icao, we will see duplicates for identical ap-pairs. We merge them.

In [21]:
to_be_merged = new_oag18[new_oag18.duplicated(subset = ['ac_code_icao','dep_ap_code', 'arr_ap_code','dep_ap_name', 'arr_ap_name', 'gc_distance'], keep=False)].sort_index()
grouped = to_be_merged.groupby(['ac_code_icao','dep_ap_code', 'arr_ap_code'])

In [22]:
for ac_group in grouped.groups.keys():
    grouped.get_group(ac_group).index[0]
    idx_first_entry = grouped.get_group(ac_group).index[0]
    idx_other_entries = grouped.get_group(ac_group).index[1:]
    freq_sum = grouped.get_group(ac_group).sum()['frequency_2018']
    fkm_sum = grouped.get_group(ac_group).sum()['fkm_2018']

    # We update the summed up freq and fkm and set it as new value for the 1st entry
    new_oag18.at[idx_first_entry, 'frequency_2018'] = freq_sum
    new_oag18.at[idx_first_entry, 'fkm_2018'] = fkm_sum
    
    # We delete all duplicated values except the first one (which is the one with the updated information)
    new_oag18.drop_duplicates(subset=['ac_code_iata','dep_ap_code', 'arr_ap_code','dep_ap_name', 'arr_ap_name', 'gc_distance'], keep='first',inplace=True) 

## Final processing

In [24]:
len(new_oag18)

191618

In [25]:
new_oag18.ac_code_iata.unique()

array(['100', '141', '142', '290', '313', '318', '319', '143', '320',
       '321', '32N', '32Q', '332', '333', '717', '343', '345', '346',
       '351', '359', '388', '733', '722', '73L', '734', '735', '339',
       'AT7', '736', '788', '738', 'AT4', 'E95', 'F50', 'L4T', '739',
       'SF3', '73G', 'F70', 'E70', '744', '74H', '752', '753', '762',
       '763', '764', '772', '773', '77L', '77W', '789', '7M8', '781',
       'A32', 'A58', '7M9', 'AN6', 'A81', 'AB4', 'AB6', 'AN4', 'AR8',
       'AR1', 'BE1', 'A38', 'A40', 'ATP', 'BE2', 'BEC', 'BNI', 'CNA',
       'CNF', 'BE4', 'C27', 'CNT', 'CR2', 'CR1', 'CR7', 'CR9', 'DHT',
       'CRK', 'CS1', 'CS3', 'CV5', 'D28', 'D38', 'DH1', 'DH2', 'DH3',
       'DH4', 'DC9', 'CN2', 'CNJ', 'DHC', 'DHP', 'DH7', 'E75', 'E7W',
       'DHL', 'EM2', 'ER3', 'ER4', 'EMB', 'EP3', 'F28', 'FRJ', 'IL9',
       'J31', 'J32', 'J41', 'M82', 'M83', 'M87', 'M88', 'GJ5', 'GRJ',
       'M90', 'MA6', 'PA1', 'PA2', 'PAG', 'PL2', 'M81', 'S20', 'SU9',
       'SWM', 'TU3',

In [26]:
len(new_oag18.ac_code_iata.unique())

136

Finally, we...
- (1) delete the two columns that have been needed for the calculations.
- (2) reset the index.
- (3) change some data types.
- (4) assign the right ac_name_oag for the new reass ac.

In [27]:
#1,2
new_oag18 = new_oag18.drop(columns=['share_of_fkm_2018', 'route']).reset_index()
new_oag18 = new_oag18.drop(columns=['index'])

In [28]:
#3
new_oag18['frequency_2018'] = new_oag18['frequency_2018'].astype(int)
new_oag18['fkm_2018'] = new_oag18['fkm_2018'].astype(float)

In [29]:
#4-1
# update the ac names that have been double icao values previously
idx_oag_name_double_icao = ac_type_designators[(~ac_type_designators['oag_name_keep'].isnull()) & \
                                               (ac_type_designators['oag_name_keep'] != 'no')].index
ac_type_designators.at[idx_oag_name_double_icao,'ac_name_oag'] = \
                                                ac_type_designators.loc[idx_oag_name_double_icao,'oag_name_keep']

In [30]:
#4-2
ac_iata_to_oag_lookup = ac_type_designators.set_index('ac_code_iata').to_dict()['ac_name_oag']
dict(list(ac_iata_to_oag_lookup.items())[10:13]) # preview
new_oag18.at[:,'ac_name_oag'] = new_oag18['ac_code_iata'].replace(ac_iata_to_oag_lookup)
new_oag18.head()

,ac_code_iata,ac_name_oag,arr_ap_code,arr_ap_name,arr_ap_cty,dep_ap_code,dep_ap_name,dep_ap_cty,gc_distance,frequency_2018,fkm_2018,ac_code_icao
0,100,Fokker 100,ADE,Aden,YE,JIB,Djibouti,DJ,248.203212,58,14395.786303,F100
1,100,Fokker 100,AMS,Amsterdam,NL,BRU,Brussels Airport,BE,157.553624,88,13864.718918,F100
2,100,Fokker 100,AMS,Amsterdam,NL,MUC,Munich International Airport,DE,676.777570,332,224690.153347,F100
3,100,Fokker 100,AZD,Yazd,IR,BND,Bandar Abbas,IR,559.459639,91,50910.827163,F100
4,100,Fokker 100,BND,Bandar Abbas,IR,AZD,Yazd,IR,559.459639,91,50910.827163,F100


In [31]:
#4: Check whether all oag names have been replaced
len(new_oag18[new_oag18['ac_name_oag'] == 'new_reass_ac'])

0

In [32]:
# Delete ICAO code
new_oag18 = new_oag18.drop(columns='ac_code_icao')

In [33]:
new_oag18.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191618 entries, 0 to 191617
Data columns (total 11 columns):
ac_code_iata      191618 non-null object
ac_name_oag       191618 non-null object
arr_ap_code       191618 non-null object
arr_ap_name       191618 non-null object
arr_ap_cty        191618 non-null object
dep_ap_code       191618 non-null object
dep_ap_name       191618 non-null object
dep_ap_cty        191618 non-null object
gc_distance       191618 non-null float64
frequency_2018    191618 non-null int32
fkm_2018          191618 non-null float64
dtypes: float64(2), int32(1), object(8)
memory usage: 15.4+ MB


In [34]:
print('By reassigning of aircraft, the number of entries has increased by {:.2f}% from {} to {} entries.'.format(len(new_oag18)/len(oag18)*100-100,len(oag18), len(new_oag18)))

By reassigning of aircraft, the number of entries has increased by 6.11% from 180578 to 191618 entries.


In [36]:
new_oag18.to_csv('intermediary_save_of_new_oag18.csv')

## Upload changed OAG data set

In [35]:
# Checking the maximum character length in columns:
new_oag18.ac_name_oag.str.len().max()

64

In [37]:
%pg_sql set role "NDA-OAG-full"

SUCCESS: query did not return any data


<cursor object at 0x000001E2E7B53E48; closed: 0>

In [38]:
%%pg_sql cur
drop table if exists "OAG"."2018_schedule_ps_conform";

CREATE TABLE "OAG"."2018_schedule_ps_conform" 
("ac_code_iata" varchar(3), 
 "ac_name_oag" varchar(64), 
 "arr_ap_code" varchar(3),
 "arr_ap_name" varchar(39),
 "arr_ap_cty" varchar(2),
 "dep_ap_code" varchar(3),
 "dep_ap_name" varchar(39),
 "dep_ap_cty" varchar(2),
 "gc_distance" float,
 "frequency_2018" int,
 "fkm_2018" float
);

COMMENT ON
COLUMN "OAG"."2018_schedule_ps_conform"."ac_code_iata" IS 'IATA aircraft code';
COMMENT ON
COLUMN "OAG"."2018_schedule_ps_conform"."ac_name_oag" IS 'OAG aircraft name';
COMMENT ON
COLUMN "OAG"."2018_schedule_ps_conform"."arr_ap_code" IS 'IATA aiport code';
COMMENT ON
COLUMN "OAG"."2018_schedule_ps_conform"."arr_ap_name" IS 'IATA aiport name';
COMMENT ON
COLUMN "OAG"."2018_schedule_ps_conform"."arr_ap_cty" IS  'IATA airport country';
COMMENT ON
COLUMN "OAG"."2018_schedule_ps_conform"."dep_ap_code" IS 'IATA aiport code';
COMMENT ON
COLUMN "OAG"."2018_schedule_ps_conform"."dep_ap_name" IS 'IATA aiport name';
COMMENT ON
COLUMN "OAG"."2018_schedule_ps_conform"."dep_ap_cty" IS 'IATA airport country';
COMMENT ON
COLUMN "OAG"."2018_schedule_ps_conform"."gc_distance" IS 'great-circle distance for this segment';
COMMENT ON
COLUMN "OAG"."2018_schedule_ps_conform"."frequency_2018" IS 'number of flights for this segment in 2018';
COMMENT ON
COLUMN "OAG"."2018_schedule_ps_conform"."fkm_2018" IS 'flight kilometers for this segment (gc_distance * frequency)';

COMMIT;

SUCCESS: query did not return any data
 cursor object as 'cur'


In [39]:
%pg_copy new_oag18 OAG.2018_schedule_ps_conform

  waring: green-mode temporarily deactivated (interrupt won't abort the import)  green mode reactivated

In [40]:
%pg_disconnect